## 层和块

在计算机视觉中广泛流行的ResNet-152架构就有数百层， 这些层是由层组（groups of layers）的重复模式组成。

块（block）可以描述单个层、由多个层组成的组件或整个模型本身。



从编程的角度来看，块由类（class）表示。 它的任何子类都必须定义一个将其输入转换为输出的前向传播函数， 并且必须存储任何必需的参数。 注意，有些块不需要任何参数。 最后，为了计算梯度，块必须具有反向传播函数。 在定义我们自己的块时，由于自动微分（在 2.5节 中引入） 提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0505, -0.0474,  0.0757,  0.1586,  0.2414,  0.0592, -0.0632, -0.0161,
          0.2122,  0.1710],
        [ 0.0819, -0.0512,  0.0100,  0.2408,  0.1187,  0.0768, -0.0035, -0.0966,
          0.1796,  0.3016]], grad_fn=<AddmmBackward0>)

## 自定义块


In [4]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):        # 继承nn.model类
    def __init__(self) -> None:
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params  ,net(x)相当于调用__call__方法，虽然在这里没有实现，也可以调用
        super().__init__()  # 调用MLP的父类Module的构造函数来执行必要的初始化。
        self.hidden = nn.Linear(20,256)     # 隐藏层
        self.out = nn.Linear(256,10)        # 输出层
    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self,X):
         # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
         return self.out(F.relu(self.hidden(X)))

# 实例化
net = MLP()
print(X.shape)
net(X)

torch.Size([2, 20])


tensor([[-0.0044, -0.1676, -0.1447,  0.0454,  0.0049, -0.0062,  0.3201,  0.0645,
          0.0580, -0.0283],
        [-0.0604, -0.2061, -0.1676,  0.0732, -0.1363, -0.0512,  0.2932,  0.1524,
          0.0736,  0.1109]], grad_fn=<AddmmBackward0>)

## 自定义顺序快

In [5]:
class MySequential(nn.Module):
    def __init__(self,*args) -> None:
        super().__init__()
        for idx,module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict  self._modules: Dict[str, Optional['Module']] = OrderedDict()        
            self._modules[str(idx)] = module
    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)      #MySequential的用法与之前为Sequential类编写的代码相同 

tensor([[ 0.0039, -0.1294,  0.0252,  0.1198,  0.0634, -0.2415,  0.0104, -0.1573,
          0.0184,  0.3332],
        [-0.0654, -0.4052,  0.0707,  0.2182,  0.1216, -0.2018, -0.0186, -0.0924,
          0.1075,  0.2124]], grad_fn=<AddmmBackward0>)

## 在前向传播中执行代码

Sequential类使模型构造变得简单，允许我们组合新的架构，而不必定义自己的类。

并不是所有架构都是简单的顺序架构，当需要更强的灵活性时，我们需要定义自己的块

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=True)
        self.linear = nn.Linear(20,20)
    def forward(self,X):
        X = self.linear(X)
        
        # 这里相当于实现了一个隐藏层，这个隐藏层的参数是由我们自己指定的
        X = F.relu(torch.mm(X,self.rand_weight)+1)
        
        # 复用全连接层，相当于两个全连接层共享参数。对经过relu函数激活后的值在进行一次线性变换
        X = self.linear(X)
        
        # 控制流，在输出返回值之前，运行了一个while循环。在L1范数大于1的条件下，将向量除以2
        while X.abs().sum() >1:
            X /=2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1200, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1078, grad_fn=<SumBackward0>)

## 自定义层

### 1.不带参数的层

在上面我们定义了在前向传播中执行代码。这里类似。下面的CenteredLayer类要从其输入中减去均值。 要构建它，我们只需继承基础层类并实现前向传播功能。

In [3]:
import torch 
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self) -> None:     
        super().__init__()
    
    def forward(self,X):
        return X-X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [7]:
# 将自定义的层作为组件合并到更加复杂的模型中

net = nn.Sequential(nn.Linear(8,128),CenteredLayer())
Y = net(torch.rand(4,8))
Y.mean()

tensor(-6.0536e-09, grad_fn=<MeanBackward0>)

### 2.带参数的层


自定义一个全连接层

In [8]:
class MyLinear(nn.Module):
    def __init__(self, in_units:int, units:int) -> None:
        """初始化参数

        Args:
            in_units (int): 输入
            units (int): 输出
        """
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units)) # 权重矩阵
        self.bias = nn.Parameter(torch.randn(units,))       # 偏置矩阵
    def forward(self,X):
        linear = torch.matmul(X,self.weight.data)+self.bias.data
        return F.relu(linear)
linear = MyLinear(5,3)
linear.weight.data

tensor([[ 0.3271,  0.0967,  0.3938],
        [ 0.0694,  0.9654, -0.5794],
        [ 0.2892,  0.8120,  0.4311],
        [-0.4123, -0.2198,  0.6118],
        [ 1.2341,  1.3351, -0.7560]])

In [9]:
linear(torch.rand(2, 5))

tensor([[0.1755, 0.6996, 0.1179],
        [0.6051, 0.5888, 0.0000]])

In [12]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[8.7210],
        [0.0000]])